In [1]:
import requests
import json

In [4]:
path_file = '/home/eboraks/Projects/icognition_backend/data/icog_pages/bergum.medium.com%2Ffour-mistakes-when-introducing-embeddings-and-vector-search-d39478a568c5.json'
with open(path_file, 'r') as f:
    data = json.load(f)

print(type(data))

<class 'dict'>


In [6]:
data

{'url': 'https://bergum.medium.com/four-mistakes-when-introducing-embeddings-and-vector-search-d39478a568c5#',
 'title': None,
 'clean_url': 'bergum.medium.com%2Ffour-mistakes-when-introducing-embeddings-and-vector-search-d39478a568c5',
 'paragraphs': {'8': 'Representing unstructured data as embedding vectors and embedding-based retrieval (EBR) using vector search is more popular than ever. What are embeddings anyway? Roy Keyes explains it well in The shortest definition of embeddings?',
  '9': 'Embeddings are learned transformations to make data more useful',
  '10': 'In academia, this process is known as representation learning and has been a field of research for decades. By transforming the data into vectors, a language native to computers, we can make the data more useful. Take BERT for text as an example. Bidirectional Encoder Representations from Transformers (BERT).',
  '11': 'How useful the representation is, depends on how we learn this transformation and how the learned way 

In [14]:

response = requests.post("http://localhost:8889/bookmark", json=data)
print(response.json()['id'])

4


In [19]:
response = requests.post("http://localhost:8888/one_sentance", json=payload)
print(response.json())

[{'summary_text': 'A New York woman who has been married 10 times has been charged with marriage fraud.'}]


In [8]:
sentences = list(data['paragraphs'].values())
chunks = concatenate_sentences_to_chunks(sentences)

for chunk in chunks:
    summary = pszemraj_summarizer(chunk)
    print(summary[0]['summary_text'])

Your max_length is set to 1024, but you input_length is only 803. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=401)


Representing unstructured data as embedding vectors and embedding-based retrieval (EBR) using vector search is more popular than ever. The answer is better model architectures, transformer architecture, and self-supervised representation learning. Machine Learning uses machine learning to learn from big data in order to apply machine learning techniques to new data. In this post, we'll focus on text models and BERT models specifically. These are deep neural network models that can be used to rank documents without human supervision. For example, snowboarders do well when they're ranked according to relevance; for golfers, however, they do poorly when it's ranked by relevance. We also briefly delve into the use case for KNN, a distributed clustering framework designed to track how many different types of information each dataset contains.
Let us expand on the accuracy error tolerance and why that is use-case dependent. If you're building an image search service with over a billion photo

In [7]:
def concatenate_sentences_to_chunks(sentences):
  
  chunks = []
  current_chunk = ''
  for sentence in sentences:
    current_chunk += sentence.replace('\n', ' ')
    if len(current_chunk.split(' ')) >= 1000:
      chunks.append(current_chunk)
      current_chunk = ''

  chunks.append(current_chunk)
  return chunks

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [13]:
# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )     


    def postprocess(self, all_outputs):
        mid_results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy = AggregationStrategy.AVERAGE,
        )
        strings = set()
        results = []
        for kp in mid_results:
            if kp.get("word") not in strings:
                strings.add(kp.get("word"))
                results.append(kp)
        
        return results

In [33]:
def prune(kp_objects, st_model):
    keyphrases = [kp.get("word") for kp in kp_objects]
    embeddings = st_model.encode(keyphrases)

    #Compute cosine similarity between all pairs
    cos_sim = util.cos_sim(embeddings, embeddings)

    #Add all pairs to a list with their cosine similarity score
    all_sentence_combinations = []
    for i in range(len(cos_sim)-1):
        for j in range(i+1, len(cos_sim)):
            all_sentence_combinations.append([cos_sim[i][j], i, j])

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

    print("Top-5 most similar pairs:")
    for score, i, j in all_sentence_combinations:
        print("{} \t {} \t {:.4f}".format(keyphrases[i], keyphrases[j], cos_sim[i][j]))

In [14]:
model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

In [16]:
kps = extractor(data['full_text'])
for kp in kps:
    print(kp)

{'entity_group': 'KEY', 'score': 0.82937986, 'word': 'unstructured data', 'start': 13, 'end': 30}
{'entity_group': 'KEY', 'score': 0.8330004, 'word': 'embedding vectors', 'start': 34, 'end': 51}
{'entity_group': 'KEY', 'score': 0.995934, 'word': 'vector search', 'start': 94, 'end': 107}
{'entity_group': 'KEY', 'score': 0.74980867, 'word': 'embeddings', 'start': 220, 'end': 230}
{'entity_group': 'KEY', 'score': 0.38632116, 'word': 'ebr', 'start': 315, 'end': 318}
{'entity_group': 'KEY', 'score': 0.8314931, 'word': 'roy keyes', 'start': 395, 'end': 404}
{'entity_group': 'KEY', 'score': 0.5396693, 'word': 'embedding', 'start': 562, 'end': 571}
{'entity_group': 'KEY', 'score': 0.42835164, 'word': 'vectors', 'start': 572, 'end': 579}
{'entity_group': 'KEY', 'score': 0.917883, 'word': 'transformations', 'start': 783, 'end': 798}
{'entity_group': 'KEY', 'score': 0.99932146, 'word': 'academia', 'start': 827, 'end': 835}
{'entity_group': 'KEY', 'score': 0.9998516, 'word': 'representation learni

In [34]:
prune(kps, model)

Top-5 most similar pairs:
embeddings 	 embedding 	 0.9631
embedding vectors 	 embedding 	 0.8369
embedding vectors 	 embeddings 	 0.8364
vector search 	 vectors 	 0.6999
roy keyes 	 keyes 	 0.6559


In [19]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [22]:
ml = nlp('machine learning')
l = nlp('representation learning')
ml.similarity(l)

0.777390566811793

In [28]:
#Sentences are encoded by calling model.encode()
emb1 = model.encode('machine learning')
emb2 = model.encode('learning')

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.6312]])


In [1]:
from lxml import html
import re
import requests
from bs4 import BeautifulSoup

In [15]:
url = "https://medium.com/@michaelgoitein/the-playing-to-win-framework-part-iii-the-strategy-choice-cascade-279c3b5ccea3"
res = requests.get(url)
content = res.text
soup = BeautifulSoup(content, 'lxml')

In [8]:
type(soup)

bs4.BeautifulSoup

In [16]:
artilces = soup.find('article')

main_article = None
content_estimator = []
for article in artilces:
    num_contect_ele = (len(article.find_all('h1')) + len(article.find_all('p')))
    content_estimator.append(num_contect_ele)

print(max(content_estimator))
content_estimator.index(max(content_estimator)) 
                                                       

19


0

In [92]:
main_article

<div class="l"><div class="l"><span class="l"></span><section><div><div class="eh ei ej ek el em"></div><div><div class="speechify-ignore l"><div class="eo ep eq er es l"></div><div class="ab ca"><div class="ch bg dx dy dz ea"><div class="ck l"><div class="ev ab"><div aria-hidden="false" class="bl"><button aria-label="Member-only story" class="l ax ao am"><div class="h k j i d"><div><div aria-hidden="false" class="bl"><svg fill="none" height="16" viewbox="0 0 20 20" width="16"><path d="M12.4 12.77l-1.81 4.99a.63.63 0 0 1-1.18 0l-1.8-4.99a.63.63 0 0 0-.38-.37l-4.99-1.81a.62.62 0 0 1 0-1.18l4.99-1.8a.63.63 0 0 0 .37-.38l1.81-4.99a.63.63 0 0 1 1.18 0l1.8 4.99a.63.63 0 0 0 .38.37l4.99 1.81a.63.63 0 0 1 0 1.18l-4.99 1.8a.63.63 0 0 0-.37.38z" fill="#FFC017"></path></svg></div></div></div><div class="s u w ew ex q"><svg class="et eu" fill="none" height="16" viewbox="0 0 20 20" width="16"><path d="M12.4 12.77l-1.81 4.99a.63.63 0 0 1-1.18 0l-1.8-4.99a.63.63 0 0 0-.38-.37l-4.99-1.81a.62.62 0 0 1

In [5]:
header_pattern = re.compile(r"h\d")
text_elements = []
for element in main_article.find_all(["p", "h1", "h2", "h3"]):
    if (element.name == 'p' and len(element.text.split(' ')) > 8):
        text_elements.append(element.text)
    elif (header_pattern.match(element.name)  and len(element.text.split(' ')) > 3): 
        text_elements.append(element.text)
    else: 
        print(f"Skipping: {element}")

len(text_elements)

Skipping: <p class="be b bf z ey">Member-only story</p>
h1
h2
Skipping: <p class="be b hk hl bj"><a class="af ag ah ai aj ak al am an ao ap aq ar hm" data-testid="authorName" href="/@michaelgoitein?source=post_page-----279c3b5ccea3--------------------------------" rel="noopener follow">Michael Goitein</a></p>
Skipping: <p class="be b hk hl ey"><span><a class="hp hq ah ai aj ak al am an ao ap aq ar hr hs ht" href="/m/signin?actionUrl=https%3A%2F%2Fmedium.com%2F_%2Fsubscribe%2Fuser%2F68c04f19aae2&amp;operation=register&amp;redirect=https%3A%2F%2Fmedium.com%2F%40michaelgoitein%2Fthe-playing-to-win-framework-part-iii-the-strategy-choice-cascade-279c3b5ccea3&amp;user=Michael+Goitein&amp;userId=68c04f19aae2&amp;source=post_page-68c04f19aae2----279c3b5ccea3---------------------post_header-----------" rel="noopener follow">Follow</a></span></p>
Skipping: <p class="be b kb z ey"><span class="jl">--</span></p>
Skipping: <p class="be b bf z ey">Share</p>
h1
Skipping: <p class="pw-post-body-paragr

13

In [13]:
soup.find(attrs={"data-testid" : "authorName"}).text


'Michael Goitein'

In [89]:
paragraphs = artilces[0].xpath("//p")
len(paragraphs)
    

67

In [90]:
paragraphs[64].text

'About'